In [2]:
import pandas as pd 
import numpy as np 
import logging
import pickle


In [3]:
#first define a dectionary for restructure the two varaibles 
REPLACE= {
    'YearsCodePro': {'Less than 1 year': 0, 'More than 50 years': 51}, 
    'YearsCode':    {'Less than 1 year': 0, 'More than 50 years': 51}}

In [4]:
data = pd.read_csv("../data/orginal/survey_results_public.csv")
copy_data = data.copy()

In [5]:
for col , replacement in REPLACE.items():
    copy_data[col]= copy_data[col].replace(replacement).astype(np.float32)

In [6]:
print("old data")
print(data["YearsCodePro"].unique().tolist())
print ("new data")
print(copy_data["YearsCodePro"].unique().tolist())

old data
[nan, '10', '4', '5', '6', '2', '30', '9', '18', '12', '21', '1', '16', 'Less than 1 year', '15', '3', '35', '7', '8', '17', '14', '26', '25', '20', '50', '34', '11', '24', '22', '13', '31', '23', '39', '41', '27', '28', '19', '33', 'More than 50 years', '37', '29', '32', '43', '40', '38', '45', '42', '46', '36', '44', '47', '48', '49']
new data
[nan, 10.0, 4.0, 5.0, 6.0, 2.0, 30.0, 9.0, 18.0, 12.0, 21.0, 1.0, 16.0, 0.0, 15.0, 3.0, 35.0, 7.0, 8.0, 17.0, 14.0, 26.0, 25.0, 20.0, 50.0, 34.0, 11.0, 24.0, 22.0, 13.0, 31.0, 23.0, 39.0, 41.0, 27.0, 28.0, 19.0, 33.0, 51.0, 37.0, 29.0, 32.0, 43.0, 40.0, 38.0, 45.0, 42.0, 46.0, 36.0, 44.0, 47.0, 48.0, 49.0]


### Next we go to next step that i want to split the data in cols such "devType" and "LanguageHaveWorkedWith" and return list of values for each row in cols 

In [7]:
def split_answers(data_series, delimiter=";"):
    
    def is_splittable(pd_series, delimiter): 
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter): 
        return pd_series.str.split(delimiter)

    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    # Else, split each value to a list 
    modified_series = split_answer(data_series, delimiter)    
    
    # Replace NAs with empty lists 
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

In [8]:
object_cols = copy_data.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    copy_data[col] = split_answers(copy_data[col])

In [12]:
copy_data.sample(1).iloc[0]

ResponseId                                                                   3449
MainBranch                                         I am a developer by profession
Employment                      Independent contractor, freelancer, or self-em...
Country                         United Kingdom of Great Britain and Northern I...
US_State                                                                      NaN
UK_Country                                                                England
EdLevel                              Bachelor’s degree (B.A., B.S., B.Eng., etc.)
Age1stCode                                                          11 - 17 years
LearnCode                       [Other online resources (ex: videos, blogs, et...
YearsCode                                                                      21
YearsCodePro                                                                   12
DevType                                                   [Developer, full-stack]
OrgSize         

In [16]:
copy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     83439 non-null  object 
 9   YearsCode                     81641 non-null  float32
 10  YearsCodePro                  61216 non-null  float32
 11  DevType                       83439 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

# data export 

In [20]:
copy_data.to_pickle("../data/processed/1_preprocessed_df.pkl")